In [1]:
import os
os.chdir("../")
%pwd

'/home/izam/Chicken-Disease-Classification-Project'

In [2]:
# preparing the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'cnnClassifier'

In [ ]:
import tensorflow as tf

In [ ]:
class ConfigurationManager:
    """
    A class for managing configuration settings related to the components.

    """
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        prarms_filepath = PARAMS_FILE_PATH
        ) -> None:
        """
        Initializes the ConfigurationManager object.

        Args:
            config_filepath (str, optional): File path for the main configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (str, optional): File path for the parameters file. Defaults to PARAMS_FILE_PATH.
        """
        
        self.config = read_yaml(config_filepath)
        self.prams = read_yaml(prarms_filepath)
        
        create_directories([self.config.artifacts_root])


    # already there    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        """
        Retrieves the configuration for preparing callbacks.

        Returns:
            PrepareCallbacksConfig: Object containing the configuration for callbacks.
        """
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        """
        Retrieves the configuration for training.

        Returns:
            TrainingConfig: Object containing the configuration for training.
        """
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [ ]:
import time

In [ ]:

class PrepareCallback:
    """
    A class for preparing callbacks during model training.

    Attributes:
        config (PrepareCallbacksConfig): Configuration object containing callback settings.

    Methods:
        __init__: Initializes the PrepareCallback object.
        _create_tb_callbacks: Creates and returns TensorBoard callback.
        _create_ckpt_callbacks: Creates and returns ModelCheckpoint callback.
        get_tb_ckpt_callbacks: Retrieves a list of TensorBoard and ModelCheckpoint callbacks.

    Usage:
        prepare_callback_config = PrepareCallbacksConfig(...)  # Initialize with appropriate configuration
        prepare_callback = PrepareCallback(config=prepare_callback_config)
        tb_ckpt_callbacks = prepare_callback.get_tb_ckpt_callbacks()
    """
    def __init__(self, config: PrepareCallbacksConfig) -> None:
        """
        Initializes the PrepareCallback object.

        Args:
            config (PrepareCallbacksConfig): Configuration object containing callback settings.
        """
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        """
        Creates and returns TensorBoard callback.

        Returns:
            tf.keras.callbacks.TensorBoard: TensorBoard callback.
        """
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        """
        Creates and returns ModelCheckpoint callback.

        Returns:
            tf.keras.callbacks.ModelCheckpoint: ModelCheckpoint callback.
        """
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        """
        Retrieves a list of TensorBoard and ModelCheckpoint callbacks.

        Returns:
            list: List containing TensorBoard and ModelCheckpoint callbacks.

        Usage:
        tb_ckpt_callbacks = prepare_callback.get_tb_ckpt_callbacks()
        """
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
# training components
class Training:
    """
    Class for managing training procedures.

    Args:
        config (TrainingConfig): Configuration object for training.

    Methods:
        get_base_model: Load the base model for training.
        train_valid_generator: Set up training and validation data generators.
        save_model: Save the trained model to a specified path.
        train: Perform the training process using the configured parameters and callbacks.
    """
    def __init__(self, config: TrainingConfig):
        """
        Initializes the Training object.

        Args:
            config (TrainingConfig): Configuration object for training.
        """
        self.config = config
    
    def get_base_model(self):
        """
        Load the base model for training.
        """
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):
        """
        Set up training and validation data generators.
        """
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Save the trained model to a specified path.

        Args:
            path (Path): The path where the model will be saved.
            model (tf.keras.Model): The trained model to be saved.
        """
        model.save(path)


    def train(self, callback_list: list):
        """
        Perform the training process using the configured parameters and callbacks.

        Args:
            callback_list (list): List of callbacks to be applied during training.
        """
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [ ]:
# pipeline
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

Model training completed successfully